Use [geotiler](https://github.com/wrobell/geotiler/) to download map data and plot some points on it.
The code is based on the [geotiler matplotlib example](https://github.com/wrobell/geotiler/blob/master/examples/ex-matplotlib.py).

In [1]:
import matplotlib.pyplot as plt
%matplotlib inline
import numpy
import gpxpy
import mplleaflet
import glob
import os
import pandas

plt.rcParams['figure.figsize'] = (16, 9)  # Size up figures a bit

In [2]:
#import geotiler
#bbox = 11.78560, 46.48083, 11.79067, 46.48283
#mm = geotiler.Map(extent=bbox, zoom=18)
#img = geotiler.render_map(mm)

In [3]:
#x0, y0 = 11.78816, 46.48114 # http://www.openstreetmap.org/search?query=46.48114%2C11.78816
#x1, y1 = 11.78771, 46.48165 # http://www.openstreetmap.org/search?query=46.48165%2C11.78771
#points = ((x0, y0), (x1, y1))
#x, y = zip(*(mm.rev_geocode(p) for p in points))
#plt.scatter(x, y, c='red', edgecolor='none', s=100, alpha=0.5)
#plt.imshow(img)
#plt.show()

The code below is based on [this python4oceanographers blog post](https://ocefpaf.github.io/python4oceanographers/blog/2014/08/18/gpx/).

In [4]:
def load_run_data(gpx_path, filter=""):
    gpx_files = glob.glob(os.path.join(gpx_path, filter + "*.gpx"))
    run_data = []
    for file_idx, gpx_file in enumerate(gpx_files): 
        gpx = gpxpy.parse(open(gpx_file, 'r'))
        # Loop through tracks
        for track_idx, track in enumerate(gpx.tracks):
            track_name = track.name
            track_time = track.get_time_bounds().start_time
            track_length = track.length_3d()
            track_duration = track.get_duration()
            track_speed = track.get_moving_data().max_speed
            
            for seg_idx, segment in enumerate(track.segments):
                segment_length = segment.length_3d()
                for point_idx, point in enumerate(segment.points):
                    run_data.append([file_idx, os.path.basename(gpx_file), track_idx, track_name, 
                                     track_time, track_length, track_duration, track_speed, 
                                     seg_idx, segment_length, point.time, point.latitude, 
                                     point.longitude, point.elevation, segment.get_speed(point_idx)])
    return run_data

In [5]:
data = load_run_data(gpx_path='gpxtracks', filter="")
df = pandas.DataFrame(data, columns=['File_Index', 'File_Name', 'Index', 'Name',
                                     'Time', 'Length', 'Duration', 'Max_Speed',
                                     'Segment_Index', 'Segment_Length', 'Point_Time', 'Point_Latitude',
                                     'Point_Longitude', 'Point_Elevation', 'Point_Speed'])

df.head()

,File_Index,File_Name,Index,Name,Time,Length,Duration,Max_Speed,Segment_Index,Segment_Length,Point_Time,Point_Latitude,Point_Longitude,Point_Elevation,Point_Speed
0,0,0.gpx,0,2016-12-09 13:57:01,2016-12-09 12:57:01,2865.836836,483,9.251168,0,2865.836836,2016-12-09 12:57:01,46.945007,7.429614,536.564941,NaN
1,0,0.gpx,0,2016-12-09 13:57:01,2016-12-09 12:57:01,2865.836836,483,9.251168,0,2865.836836,2016-12-09 12:57:02,46.945055,7.429586,540.644531,9.099847
2,0,0.gpx,0,2016-12-09 13:57:01,2016-12-09 12:57:01,2865.836836,483,9.251168,0,2865.836836,2016-12-09 12:57:03,46.945151,7.429544,541.273682,10.243475
3,0,0.gpx,0,2016-12-09 13:57:01,2016-12-09 12:57:01,2865.836836,483,9.251168,0,2865.836836,2016-12-09 12:57:04,46.945223,7.429530,536.550049,8.525432
4,0,0.gpx,0,2016-12-09 13:57:01,2016-12-09 12:57:01,2865.836836,483,9.251168,0,2865.836836,2016-12-09 12:57:05,46.945287,7.429538,539.381104,6.911977


In [ ]:
for i in range(8):
    plt.plot(df['Point_Longitude'][df['File_Index']==i],
             df['Point_Latitude'][df['File_Index']==i],
             color='black', linewidth=7)
mplleaflet.display(tiles='stamen_wc')

The code below does the same with [folium](https://github.com/python-visualization/folium), based on [this blog post](https://blog.dominodatalab.com/creating-interactive-crime-maps-with-folium/).

In [ ]:
import folium

durchschnitt = (numpy.mean(df['Point_Latitude']),numpy.mean(df['Point_Longitude']))
map = folium.Map(location=durchschnitt, zoom_start=14,  tiles='Stamen Toner')

for each in df.iterrows():  
    folium.CircleMarker(location=[each[1][11],each[1][12]], radius=3,
                        popup='asdf', color='red',
                        fill_color='red').add_to(map)

In [ ]:
map